# Real Time Face Recognition using Webcam

Objective: To recognise the face of a person showing on a webcam.

Pre-trained model: Keras-OpenFace (open source keras implementation of the OpenFace)

Inspiration Credit: FaceNet & DeepFace

In [1]:
# Import the libraries
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import glob

from utils import LRN2D
import utils

from inception_blocks_v2 import inception_block_3a
from inception_blocks_v2 import inception_block_3b
from inception_blocks_v2 import inception_block_3c
from inception_blocks_v2 import inception_block_4a
from inception_blocks_v2 import inception_block_4e
from inception_blocks_v2 import inception_block_5a
from inception_blocks_v2 import inception_block_5b
import inception_blocks_v2

%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
# Construct neural network model (based on FaceNet's Inception model)
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inceptions
inception_3a = inception_block_3a(x)
inception_3b = inception_block_3b(inception_3a)
inception_3c = inception_block_3c(inception_3b)
inception_4a = inception_block_4a(inception_3c)
inception_4e = inception_block_4e(inception_4a)
inception_5a = inception_block_5a(inception_4e)
inception_5b = inception_block_5b(inception_5a)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)

# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)


Instructions for updating:
Colocations handled automatically by placer.


In [3]:
# Load pretrained weights from csv files (which was exported from Openface torch model)
weights = utils.weights
weights_dict = utils.load_weights()

# Set layer weights of the model
for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

In [8]:
# Recognise face real time from webcam
def image_to_embedding(image, model):
    
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

def recognize_face(face_image, input_embeddings, model):
    
    embedding = image_to_embedding(face_image, model)

    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        #print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))       
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.68:
        return str(name)
    else:
        return None

def create_input_image_embeddings():
    
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

def recognize_faces_in_cam(input_embeddings):
    
    cv2.namedWindow("Real Time Face Recognition")
    vc = cv2.VideoCapture(0)
   
    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape
       
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h
           
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Real Time Face Recognition", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)